In [2058]:
import os
import json
from IPython.display import HTML, display
try:
    (notebook_dir, e)
except:
    e = "a --- b"
    notebook_dir = "."

In [2059]:
padding = {
    "ks_": -100,
    "ts_": -380,
    "ka_": -280,
    "ko_": -390,
    "ta_": -250,
    "to_": -250,
    "tb_": -250,
}

### Nommage des fichiers : 
* Lettre concernée _ left ou right ou lettertoconnect_right ; 
* t_and_a_left pour les clusters
* ks_above_t
* **hz**_440 (ligne horizontale de largeur 440)
* vt_80_220 (vertical debut et fin) ; vt_**n**160_220 pour du négatif

In [2060]:
fea = '''
feature ccmp {
    script DFLT;
    language dflt required;
    script latn;
    language dflt required;

    lookup ETAPE_1_CHOIX {  # Choix forme contextuelle de base

        # Lettre F

        # Lettre B

        sub [o n f] b' by __ob;
        sub [s k] b' by __sb;
        sub b' by __ab;

        # Lettre U

        sub [n] u' by u;  # Keep the regular u, ignore next letter
        sub u' [a e p o b c y i n] by _u;

    } ETAPE_1_CHOIX;

    # 2. A partir de la on ne manipule que des glyphs avec des points d'attache constants
    # On traite toutes les paires de lettres deux a deux 
    # en assumant qu'elles sont en milieu de mot
    # Ca inclu les formes f / n / etc;

    lookup ETAPE_2_COMPOSANTS {  # Decoupe tous les caracteres en composants

        # A E P | O Y

        sub a by a_ _a;
        sub o by o_ _o;
        sub e by a_ _e;
        sub p by a_ _p;
        sub y by o_ _y;

        # B
        
        sub __ob by o_ _b;
        sub __ab by b_ _o;
        sub __sb by _sb_ _o;

        # U

        # S

        # sub s by s_ _s;

    } ETAPE_2_COMPOSANTS;

    # 3. Fusionne les consonnes et voyelles en assumant qu'elles sont en début ou milieu

    lookup ETAPE_3_FUSION {  # Fusionne les composants des que possible

        # AO OA

        sub _a o_ by _ao_;
        sub _o a_ by _oa_;

        sub _oa_ a_ by _oa_;
        sub _ao_ o_ by _ao_;

        # S

        # sub _s a_ by _sa_;

        # K

        # sub k s_ by ks_;

    } ETAPE_3_FUSION;

    # 4. Modifie et positionne les accents

    # lookup ETAPE_5_ACCENTS {  # Modifie et positionne les accents
    # } ETAPE_5_ACCENTS;

    # 5. On applique les formes purement esthétiques (connexions avec les lignes horizontales...)

    # lookup ETAPE_6_ESTHETIQUE {  # Applique les formes purement esthétiques
        
        # N F CONNECT FORMS

        # sub n a_' by _ofa_ a_; ...

        # U B ALTERNATES ... or something

        # sub n u' by _nu;

        # CODAS (eb, -k, -t, ...) << le positionnement de "i" n'est pas affecté donc c un vrai truc esthétique

    # } ETAPE_6_ESTHETIQUE;

    

    # TOUT CE QUI SE TROUVE CI DESSOUS DOIT ETRE FUSIONNER PLUS HAUT

    lookup consonnant_by_vowel {
        sub s' [a_ o_ _sb_ c] by s_;

        sub f' [a_ c u _u _sb_] by af;
        sub k' [a_ c u _u _sb_] by ka_;

        sub f' [o_] by of;
        sub k' [o_] by ko_;
    } consonnant_by_vowel;

    lookup consonnant_by_preceding_vowel {
        sub [_a _p] f' by af;
        sub [_o _e] f' by of;
    } consonnant_by_preceding_vowel ;

    lookup vowel_by_consonnant {

        sub s_ a_' by _sa_;
        sub s_ o_' by _so_ o_;
        sub s_ c' by _sc;
        
        sub [n of] [a_]' by _ofa_ [a_];  # ofa marker

        sub _a' [n of] by _a _ofa_;  # mwm
        sub _p' [n of] by _p _ofa_;

        sub [n of] c' [n of] by _cnc_ c _cnc_;  # c
        sub c' [n of] by c _cnc_;
        sub [n of] c' by _cnc_ c;

        # U junctions with horizontal consonants
        sub ka_ _u' [n of] by _ofa_ _ku;
        sub ka_ _u' by _ku;
        sub [n of] u' by _ofa_ _afu;
        sub [ka_ af] u' by _afu;
        sub _u' n by ufa_ _ofa_;
        sub _u' af by ufa_;

        sub [n af] o_' by _afo_ o_;  # afo marker
        sub _o' [n af] by _o _afo_;
        
        sub _e' [n of s] by _en_;
        sub _b' [n of s] by _bn_;

    } vowel_by_consonnant;

    lookup collapse_consonnants {
        sub k s by ks;
        sub k s_ by ks_;
        sub t s_ by ts_;
        sub t a_ by ta_;
        sub t o_ by to_;
        sub t b_ by tb_;
        sub t c by tc;
    } collapse_consonnants;

    lookup consonnant_codas {
        sub [_a n af] s' by _as;
        sub [_o of _en_] s' by _os;
    } consonnant_codas;

    lookup vowel_by_consonnant_codas {
        sub _o' _os by _os_;
    } vowel_by_consonnant_codas;
} ccmp;
'''

In [2061]:
fea += '''
feature kern {
    script DFLT;
    language dflt;
    script latn;
    language dflt;

    pos [_a _o] [a_ o_] -80;  # VOYELLE A DROITE

    pos [_so_] [o_] -80;  # S A GAUCHE
    pos [s_ ks_] [_sa_ _so_ _sc _sb_] -80;

    pos [_a _o n of af _os_ _en_] [_as _os] -80;  # S A DROITE

    # horizontal consonnants
    pos [_a _o _p] [_ofa_ _afo_] -320;  # N A DROITE
    pos [_a _o _p] [af of n] -70;
    
    pos [_ofa_ _afo_] [a_ o_ _afu] -320;  # N A GAUCHE
    pos [af of n] [a_ o_] -70;

    pos _afu _ofa_ -250;
    pos ufa_ _ofa_ -260;

    pos c _cnc_ -200;
    pos _cnc_ c -200;

    pos [_ofa_ _afo_ _cnc_ _afu] [af of n] -70;
    pos [af of n] [_ofa_ _afo_ _cnc_ _afu] -70;

    pos ka_ c -40;

    # VOYELLES DEUX A DEUX

    # COMPOSANTS S

    # ELEMENTS HORIZONTAUX ESTHETIQUES
    
} kern;
'''

In [2062]:
try: counter += 1
except: counter = 0
if not os.getcwd().endswith('bin'): 
    notebook_dir = os.getcwd()  # Actual working directory
    %cd ".\spetekkimyo\ffpython\bin\"
json.dump(padding, open(f'{notebook_dir}\\spetekkimyo\\input\\padding.json', 'w'))
with open(f'{notebook_dir}\\spetekkimyo\\input\\features.fea', 'w') as f: f.write(fea)
!"ffpython.exe" "..\..\generate.py" "{notebook_dir}\output\test.otf"

Imported 62 glyphs: a af a_ b b_ c e f i k ka_ ko_ ks ks_ n o of o_ p s s_ t ta_ tb_ tc tka_ tko_ tks_ to_ ts_ u ufa_ y _a _afo_ _afu _ao_ _as _b _bn_ _cnc_ _e _en_ _fc _ku _nc _o _oa_ _ofa_ _ofu_ _os _os_ _p _sa_ _sb_ _sc _so_ _u _y __ab __ob __sb
Imported features
Font generated at d:\Github\spetekkimyo\output\test.otf


In [2063]:
e = """
<span class="ss">o oa oaoa oaa tbfbsbnofas</span> pb
"""

In [2064]:
html = """<style>p {font-size: 80px;} @font-face {src:url('./output/test.otf?version=%s');font-family:'test';} .ss {font-family:'test';}</style>
<p>%s</p>""" % (counter, e)
display(HTML(html))

### Guidelines

Epaisseur du trait : 80

Largeur : **550** (275) -- consonants 320 - 440 (max d'épaisseur) - 550 (pointe droite)

Hauteur standard : 320 -- max = 730 -- low = -240

Départ : h=-12